# Hourly Echo

Tweet quote every hour.

---

In [1]:
import random
import os
import sqlite3
import getopt
import tweepy
import sys

---

### Random Quote

In [2]:
conn = sqlite3.connect('db/quotes.db')
cur = conn.cursor()

In [3]:
# fetch one random record where tweeted = 0
quote_data = cur.execute("SELECT * FROM quotes WHERE tweeted = 0 ORDER BY RANDOM() LIMIT 1").fetchone()
quote = quote_data[0]
author = quote_data[1]

### Random Background Image

In [4]:
images = os.listdir('images')
background_image = random.choice(images)

### Random Font

In [5]:
fonts = os.listdir('fonts')
font = random.choice(fonts).split(".")[0]

---

### Merge Quote and Background Image

In [6]:
!mkdir -p output

In [7]:
file = open("output/quote.srt", "w")
file.write(f"""1
00:00:00,000 --> 00:00:05,000
{quote}""")
file.close()

file = open("output/author.srt", "w")
file.write(f"""1
00:00:00,000 --> 00:00:05,000
{author}""")
file.close()

In [8]:
!sh generate_image.sh "images/{background_image}" "{font}"

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

---

### Twitter Credentials

1. Try to get the credentials passed as arguments
2. If not, try to get the credentials from the `.env` file

In [9]:
CONSUMER_KEY = ""
CONSUMER_SECRET = ""
ACCESS_TOKEN = ""
ACCESS_TOKEN_SECRET = ""


In [10]:
argumentList = sys.argv[1:]

options = "1:2:3:4:"

long_options = [
    "twitter_consumer_key=",
    "twitter_consumer_secret=",
    "twitter_access_token=",
    "twitter_access_token_secret=",
]

try:
    arguments, values = getopt.getopt(argumentList, options, long_options)

    for currentArgument, currentValue in arguments:

        if currentArgument in ["-1", "--twitter_consumer_key"]:
            CONSUMER_KEY = currentValue
        elif currentArgument in ["-2", "--twitter_consumer_secret"]:
            CONSUMER_SECRET = currentValue
        elif currentArgument in ["-3", "--twitter_access_token"]:
            ACCESS_TOKEN = currentValue
        elif currentArgument in ["-4", "--twitter_access_token_secret"]:
            ACCESS_TOKEN_SECRET = currentValue

except getopt.error as err:
    """You may see an error like 'option --ip not recognized' if executed from Jupyter Notebook"""
    print(str(err))


option --ip not recognized


In [11]:
print("Consumer Key:", CONSUMER_KEY)
print("Consumer Secret:", CONSUMER_SECRET)
print("Access Token:", ACCESS_TOKEN)
print("Access Token Secret:", ACCESS_TOKEN_SECRET)


Consumer Key: 
Consumer Secret: 
Access Token: 
Access Token Secret: 


In [12]:
%load_ext dotenv
%dotenv

if CONSUMER_KEY == "" or CONSUMER_SECRET == "" or ACCESS_TOKEN == "" or ACCESS_TOKEN_SECRET == "":
    if CONSUMER_KEY == "":
        CONSUMER_KEY = os.getenv("TWITTER_CONSUMER_KEY")
    if CONSUMER_SECRET == "":
        CONSUMER_SECRET = os.getenv("TWITTER_CONSUMER_SECRET")
    if ACCESS_TOKEN == "":
        ACCESS_TOKEN = os.getenv("TWITTER_ACCESS_TOKEN")
    if ACCESS_TOKEN_SECRET == "":
        ACCESS_TOKEN_SECRET = os.getenv("TWITTER_ACCESS_TOKEN_SECRET")
    print("Consumer Key:", CONSUMER_KEY)
    print("Consumer Secret:", CONSUMER_SECRET)
    print("Access Token:", ACCESS_TOKEN)
    print("Access Token Secret:", ACCESS_TOKEN_SECRET)


Consumer Key: 5AO2rtqAMJxAvAtMkpiAYJIBt
Consumer Secret: ZPPUa5eCBHiY3Q7VjkE1StT2MnTRdG384jzicAGX7B27S4ECkq
Access Token: 1596233231384588288-97xCedD9MCaeaQZfcDw4CI6mFwVvfO
Access Token Secret: Jt3AhEMYW7WI43v1M6b2oLBcw8KeHZTiaVUkGeZqiItQh


---

In [13]:
if CONSUMER_KEY == "" or CONSUMER_KEY == None:
    print("No consumer key")
if CONSUMER_SECRET == "" or CONSUMER_SECRET == None:
    print("No consumer secret")
if ACCESS_TOKEN == "" or ACCESS_TOKEN == None:
    print("No access key")
if ACCESS_TOKEN_SECRET == "" or ACCESS_TOKEN_SECRET == None:
    print("No access secret")
else:
    # upload image to twitter
    auth = tweepy.OAuthHandler(
        CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

    api = tweepy.API(auth)

    media = api.media_upload(filename="output/output.png")
    print(media.media_id)

    tweet = api.update_status(status="", media_ids=[media.media_id])
    print(tweet.id)

    cur.execute("UPDATE quotes SET tweeted = 1 WHERE quote = ?", (quote,))
    conn.commit()
    conn.close()


1596244973401186304
1596244978300178432


---

<p align="center">Made with 💖 by 9 Bit of Code</p>

---